<a href="https://colab.research.google.com/github/dsz1910/intracranial-aneurysm--ai/blob/main/aneurysm_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install SimpleITK

In [ ]:
!pip install k3d

In [ ]:
import k3d
import pandas as pd
import numpy as np
import os
import glob
import SimpleITK as sitk
import json
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

seg_with_loc = ['1.2.826.0.1.3680043.8.498.10035643165968342618460849823699311381',
 '1.2.826.0.1.3680043.8.498.10076056930521523789588901704956188485',
 '1.2.826.0.1.3680043.8.498.10188636688783982623025997809119805350',
 '1.2.826.0.1.3680043.8.498.10410600166004340343973545138447283460',
 '1.2.826.0.1.3680043.8.498.10540586847553109495238524904638776495',
 '1.2.826.0.1.3680043.8.498.10929608782694347957516071062422315982',
 '1.2.826.0.1.3680043.8.498.10935907012185032169927418164924236382',
 '1.2.826.0.1.3680043.8.498.11140496970152788589837488009637704168',
 '1.2.826.0.1.3680043.8.498.11163718560814217911019576488539324434',
 '1.2.826.0.1.3680043.8.498.11422928060228360802778018026859204182',
 '1.2.826.0.1.3680043.8.498.11447542941959800581541313722844637822',
 '1.2.826.0.1.3680043.8.498.11504459395565711149380261095223705023',
 '1.2.826.0.1.3680043.8.498.11557464859397815362951522785245632020',
 '1.2.826.0.1.3680043.8.498.11624217734793256238140178687655335066',
 '1.2.826.0.1.3680043.8.498.11639720015527164474926997755882681707',
 '1.2.826.0.1.3680043.8.498.11641438607169452758239778414614826230',
 '1.2.826.0.1.3680043.8.498.11821446229980500432989393232863242415',
 '1.2.826.0.1.3680043.8.498.11924949819899884502738782576851659426',
 '1.2.826.0.1.3680043.8.498.11938739392606296532297884225608408867',
 '1.2.826.0.1.3680043.8.498.11999987145696510072091906561590137848',
 '1.2.826.0.1.3680043.8.498.12132622846836853200891705613461466627',
 '1.2.826.0.1.3680043.8.498.12180351938456969219537687190067731477',
 '1.2.826.0.1.3680043.8.498.12226380705607315060235896835122737788',
 '1.2.826.0.1.3680043.8.498.12271269630687930751200307891697907423',
 '1.2.826.0.1.3680043.8.498.12427930128533148989436011949311706348',
 '1.2.826.0.1.3680043.8.498.12709802490782896897031103447163443069',
 '1.2.826.0.1.3680043.8.498.12773309706735630359315214846273921394',
 '1.2.826.0.1.3680043.8.498.12792960392435514526913217158720555996',
 '1.2.826.0.1.3680043.8.498.12812390336793304037901571645929430100',
 '1.2.826.0.1.3680043.8.498.12873050136415197430227722045995986358',
 '1.2.826.0.1.3680043.8.498.12888459003897616398890411591973176636',
 '1.2.826.0.1.3680043.8.498.12896910506681881306246412668919668702',
 '1.2.826.0.1.3680043.8.498.12898332622076283462996059479076432725',
 '1.2.826.0.1.3680043.8.498.12904246053955178641505906243733756576',
 '1.2.826.0.1.3680043.8.498.12914952223659958493995413641114579279',
 '1.2.826.0.1.3680043.8.498.13128656559176299272467358793386537400',
 '1.2.826.0.1.3680043.8.498.13359737970612926494907045108541390310',
 '1.2.826.0.1.3680043.8.498.13789305723712362238118274295587312089',
 '1.2.826.0.1.3680043.8.498.14375161350968928494386548917647435597',
 '1.2.826.0.1.3680043.8.498.15111820005882064793593034423469604305',
 '1.2.826.0.1.3680043.8.498.15412988336827906186857260013885503248',
 '1.2.826.0.1.3680043.8.498.15777485274723969278718374949878560903',
 '1.2.826.0.1.3680043.8.498.16386250344855221757144432829845114733',
 '1.2.826.0.1.3680043.8.498.16984390277144742906667579449023180512',
 '1.2.826.0.1.3680043.8.498.17415277997649872560329721717694101082',
 '1.2.826.0.1.3680043.8.498.19915189891686122627071348069843885714',
 '1.2.826.0.1.3680043.8.498.21004106426734635526381567602936015568',
 '1.2.826.0.1.3680043.8.498.23047023542526806696555440426928375679',
 '1.2.826.0.1.3680043.8.498.23055827202917388669053993133576833763',
 '1.2.826.0.1.3680043.8.498.24023896361071846724104915533800547445',
 '1.2.826.0.1.3680043.8.498.24587963869128721940158079207224095554',
 '1.2.826.0.1.3680043.8.498.24941924992372724575490063788348447936',
 '1.2.826.0.1.3680043.8.498.27693546360513068451517048347207987807',
 '1.2.826.0.1.3680043.8.498.27857528510177554953207997404329765760',
 '1.2.826.0.1.3680043.8.498.28722601444191262075880952461419085326',
 '1.2.826.0.1.3680043.8.498.31897325247898403027455884342546675049',
 '1.2.826.0.1.3680043.8.498.32250259987224176174516959348681094310',
 '1.2.826.0.1.3680043.8.498.34439485184360273751379923196589017042',
 '1.2.826.0.1.3680043.8.498.35327124657045713676192746001247576881',
 '1.2.826.0.1.3680043.8.498.35378146560080702211693278243609271022',
 '1.2.826.0.1.3680043.8.498.36205761227502095958293403225062705137',
 '1.2.826.0.1.3680043.8.498.36516744229109249667702200145077143886',
 '1.2.826.0.1.3680043.8.498.36563348911961346172279351080943665664',
 '1.2.826.0.1.3680043.8.498.36928611823925733133253145871406408988',
 '1.2.826.0.1.3680043.8.498.37086262716517957668471635372810376638',
 '1.2.826.0.1.3680043.8.498.38245669369430321272819874468980907728',
 '1.2.826.0.1.3680043.8.498.38904475631578710113273863766282479811',
 '1.2.826.0.1.3680043.8.498.39640919070091958876744231048011388614',
 '1.2.826.0.1.3680043.8.498.40402571428459178954472078378902050472',
 '1.2.826.0.1.3680043.8.498.42092450058597943280470345107435382425',
 '1.2.826.0.1.3680043.8.498.42672154202952548010999212369080894652',
 '1.2.826.0.1.3680043.8.498.42933230680553480084056393591634621848',
 '1.2.826.0.1.3680043.8.498.43495968397556043698567120038117641587',
 '1.2.826.0.1.3680043.8.498.43536331102142701793144520859521601945',
 '1.2.826.0.1.3680043.8.498.47622062519393262272120105951011625928',
 '1.2.826.0.1.3680043.8.498.47802313478131783077762931281303667601',
 '1.2.826.0.1.3680043.8.498.47887093599897399482447594752785316358',
 '1.2.826.0.1.3680043.8.498.49640345168968922611291772802640560828',
 '1.2.826.0.1.3680043.8.498.49718418682238683779854914910561017368',
 '1.2.826.0.1.3680043.8.498.50241233088534910114736887318508484246',
 '1.2.826.0.1.3680043.8.498.50268462808449401128173812870329002342',
 '1.2.826.0.1.3680043.8.498.50275403170194436966991630938339966596',
 '1.2.826.0.1.3680043.8.498.50369188120242587742908379292729868174',
 '1.2.826.0.1.3680043.8.498.52363954882447190271251269039176558430',
 '1.2.826.0.1.3680043.8.498.53947155422591684879953627516013605305',
 '1.2.826.0.1.3680043.8.498.54865110953409154322874363435644372368',
 '1.2.826.0.1.3680043.8.498.55051557363776453883164282380323354147',
 '1.2.826.0.1.3680043.8.498.55520651046049733868642268089599441721',
 '1.2.826.0.1.3680043.8.498.56109731607412273442907651635753012241',
 '1.2.826.0.1.3680043.8.498.56479623144539472445940519727300319231',
 '1.2.826.0.1.3680043.8.498.56867346585094457716984380929416039466',
 '1.2.826.0.1.3680043.8.498.58839417089022860359638460482101293080',
 '1.2.826.0.1.3680043.8.498.61152918475243358118286003299125054478',
 '1.2.826.0.1.3680043.8.498.65011208113835286935212080363533579671',
 '1.2.826.0.1.3680043.8.498.65654303333996310125136982540737772052',
 '1.2.826.0.1.3680043.8.498.66341469849558089736451534296312923277',
 '1.2.826.0.1.3680043.8.498.67256382079119118825371537284628604044',
 '1.2.826.0.1.3680043.8.498.68161752706586485995657009830735928975',
 '1.2.826.0.1.3680043.8.498.68276712082656957005274595949315894066',
 '1.2.826.0.1.3680043.8.498.68356160898101066850726244725552676010',
 '1.2.826.0.1.3680043.8.498.68654901185438820364160878605611510817',
 '1.2.826.0.1.3680043.8.498.68709340002397343932718258443293606585',
 '1.2.826.0.1.3680043.8.498.69401690945645968072368812538918487252',
 '1.2.826.0.1.3680043.8.498.69568746915553014138135720681936366640',
 '1.2.826.0.1.3680043.8.498.70243202242722756546202582478829903758',
 '1.2.826.0.1.3680043.8.498.71796601538792777580416179841706319140',
 '1.2.826.0.1.3680043.8.498.72679260079421518845786364620483278827',
 '1.2.826.0.1.3680043.8.498.73820261697830420042473892884688067574',
 '1.2.826.0.1.3680043.8.498.75016896260047968433534297207591136672',
 '1.2.826.0.1.3680043.8.498.75294325392457179365040684378207706807',
 '1.2.826.0.1.3680043.8.498.75798029534455454939797323020706657426',
 '1.2.826.0.1.3680043.8.498.77257791208759842602760935296318202703',
 '1.2.826.0.1.3680043.8.498.79099213587801933936080747802403048718',
 '1.2.826.0.1.3680043.8.498.79942836660118710928733936389534291771',
 '1.2.826.0.1.3680043.8.498.80048101091444895066772572129871971243',
 '1.2.826.0.1.3680043.8.498.80114244849666367523293067199486077713',
 '1.2.826.0.1.3680043.8.498.80190289468142266421549927426167714158',
 '1.2.826.0.1.3680043.8.498.80461517820710375402982229582943598734',
 '1.2.826.0.1.3680043.8.498.81098958708250149437576237811675033160',
 '1.2.826.0.1.3680043.8.498.82247540847692847800462620079965863384',
 '1.2.826.0.1.3680043.8.498.82641698422464356104108563099150990855',
 '1.2.826.0.1.3680043.8.498.84908441442551598157537604822760711232',
 '1.2.826.0.1.3680043.8.498.84955070686251417902923705821409495324',
 '1.2.826.0.1.3680043.8.498.85709849872024108265120796348331660195',
 '1.2.826.0.1.3680043.8.498.86822530556046989269633487715061058236',
 '1.2.826.0.1.3680043.8.498.87794163393266428648659243169230666286',
 '1.2.826.0.1.3680043.8.498.88044882887797890422716086408658477347',
 '1.2.826.0.1.3680043.8.498.88512241250207324783783101806489145581',
 '1.2.826.0.1.3680043.8.498.88662334466087798807484415780594176763',
 '1.2.826.0.1.3680043.8.498.88739296218460643753583291722714541935',
 '1.2.826.0.1.3680043.8.498.88905360377095450551559885185901908404',
 '1.2.826.0.1.3680043.8.498.89421386426320866039573378582181968701',
 '1.2.826.0.1.3680043.8.498.89990837914171555676446644356114244393',
 '1.2.826.0.1.3680043.8.498.90015157820692758596783999454928886688',
 '1.2.826.0.1.3680043.8.498.90168683694094931217787644438845074017',
 '1.2.826.0.1.3680043.8.498.92418959634964175917370213963992652610',
 '1.2.826.0.1.3680043.8.498.92543328866053664733167983708344898988',
 '1.2.826.0.1.3680043.8.498.92773748942952645243074808740855383414',
 '1.2.826.0.1.3680043.8.498.96218477847514569819859044953648183121',
 '1.2.826.0.1.3680043.8.498.97057911327885502714270510313728134927',
 '1.2.826.0.1.3680043.8.498.97256479550884529885940791074752719030',
 '1.2.826.0.1.3680043.8.498.98123758735027035609698227781754927939',
 '1.2.826.0.1.3680043.8.498.98133633346919790888527055899070500258']

In [ ]:
prefix = '/content/drive/Shareddrives/DATA/rsna-intracranial-aneurysm-detection/'
series_path = os.path.join(prefix, 'series')
segmentations_path = os.path.join(prefix, 'segmentations')

In [ ]:
def resample_nii_to_dcm(nii_img_path, dcm_img_path):
    dcm_img = sitk.ReadImage(dcm_img_path)
    nii_img = sitk.ReadImage(nii_img_path)

    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(dcm_img)
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    resampler.SetDefaultPixelValue(0)

    resampled_nii = resampler.Execute(nii_img)

    return resampled_nii


In [ ]:
def get_affine_matrix(image):
    origin = np.array(image.GetOrigin())
    spacing = np.array(image.GetSpacing())
    direction = np.array(image.GetDirection()).reshape(3, 3)

    linear_matrix = np.dot(direction, np.diag(spacing))

    affine = np.eye(4)
    affine[:3, :3] = linear_matrix
    affine[:3, 3] = origin

    return affine

In [ ]:
def map_points(x, y, z, affine):
    inv_affine = np.linalg.inv(affine)
    vector = np.array([x, y, z, 1])

    i, j, k = np.dot(inv_affine, vector)[:3]

    return int(np.round(i)), int(np.round(j)), int(np.round(k))

In [ ]:
def get_physical_point(img, df_row):
    xy = df_row['coordinates']
    x = json.loads(xy.replace("\'", "\""))['x']
    y = json.loads(xy.replace("\'", "\""))['y']

    return x, y

In [ ]:
localizers = pd.read_csv(os.path.join(prefix,'train_localizers.csv'))
train_df = pd.read_csv(os.path.join(prefix, 'train.csv'))

In [ ]:
localizers.head()

In [ ]:
train_df['Modality'].unique()

In [ ]:
localizers.columns

In [ ]:
print(localizers.shape[0])
print(len(localizers['SeriesInstanceUID'].unique()))
len(localizers['SOPInstanceUID'].unique())

Różnica orientacji pomiędzy .nii oraz .dcm

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

nii_dcm_orient_diff = {}
corrected_nii_dcm_orient_diff = {}

for ser in seg_with_loc:
    sops = localizers.loc[localizers['SeriesInstanceUID'] == ser]

    nii_path = os.path.join(segmentations_path, f'{ser}.nii')
    nii_img = sitk.ReadImage(nii_path)
    nii_orient = nii_img.GetDirection()
    corrected_nii_orient = sitk.DICOMOrient(nii_img).GetDirection()

    for ith, row in sops.iterrows():
        slice_path = os.path.join(series_path, ser, f'{row.SOPInstanceUID}.dcm')
        dcm_img = sitk.ReadImage(slice_path)
        dcm_orient = dcm_img.GetDirection()

        nii_dcm_orient_diff[row.SOPInstanceUID] = dcm_orient != nii_orient
        corrected_nii_dcm_orient_diff[row.SOPInstanceUID] = dcm_orient != corrected_nii_orient

    '''for idx, row in sop.iterrows():
        slice_path = os.path.join(series_path, ser, f'{row.SOPInstanceUID}.dcm')
        image_2d = sitk.ReadImage(slice_path)

        dcm_orient = image_2d.GetDirection()
        #dcm_orient = sitk.DICOMOrient(image_2d)
        print(dcm_orient, nii_orient, corrected_nii_orient)

        img = sitk.GetArrayFromImage(image_2d)
        print(img.shape)
        fig, ax = plt.subplots()
        ax.imshow(img.squeeze(), cmap='gray')
        plt.show()
        break
    break'''

print(np.bincount(np.array(list(nii_dcm_orient_diff.values())).astype(int)))
print(np.bincount(np.array(list(corrected_nii_dcm_orient_diff.values())).astype(int)))

początkowo wszystkie pliki .nii mają inną orientację niż odpowiadające im pliki .dcm, a po sitk.DICOMOrient 89 obrazów .nii ma odpowiednią orientację na 184

In [ ]:
nii_example = sitk.ReadImage(os.path.join(segmentations_path, f'{seg_with_loc[0]}.nii'))
affine = get_affine_matrix(nii_example)


In [ ]:
for ser, val in corrected_nii_dcm_orient_diff.items():
    if val:
        img_path = os.path.join(segmentations_path, f'{ser}.nii')
        img = sitk.ReadImage(img_path)
        print(f'Original orientation: {img.GetDirection()}')
        print(f'Orientation after correction: {sitk.DICOMOrient(img).GetDirection()}')
        break

In [ ]:
from pathlib import Path

folder = Path('/content/drive/Shareddrives/DATA/g143_review/Tetniaki_i_histogramy_D_Szyjka')
folder.mkdir(exist_ok=True)

In [ ]:
for i, ser in enumerate(seg_with_loc):
    print(i)
    ser_modality = train_df.loc[train_df['SeriesInstanceUID'] == ser]['Modality'].iloc[0]

    sop_df = localizers.loc[localizers['SeriesInstanceUID'] == ser]
    nii_path = os.path.join(segmentations_path, f'{ser}.nii')
    nii_img = sitk.ReadImage(nii_path)
    nii_img = sitk.DICOMOrient(nii_img)

    seg_path = os.path.join(segmentations_path, f'{ser}_cowseg.nii')
    seg = sitk.ReadImage(seg_path)
    seg = sitk.DICOMOrient(seg)

    for idx, row in  sop_df.iterrows():
        slice_path = os.path.join(series_path, row.SeriesInstanceUID, f'{row.SOPInstanceUID}.dcm')
        img = sitk.ReadImage(slice_path)

        x, y = get_physical_point(img, row)
        x, y, z = np.array(img.TransformContinuousIndexToPhysicalPoint((x, y, 0)))

        nii_k, nii_j, nii_i = nii_img.TransformPhysicalPointToIndex((x, y, z))

        file_name = f'{ser}_{nii_i:04d}_{ser_modality}'

        nii_arr = sitk.GetArrayFromImage(nii_img)
        seg_arr = sitk.GetArrayFromImage(seg)

        nii_slice = nii_arr[nii_i, :, :]
        seg_slice = seg_arr[nii_i, :, :]

        p5 = np.percentile(nii_slice, 5)
        p95 = np.percentile(nii_slice, 95)

        if ser_modality == 'CTA':
            vmin = -100
            vmax = 600
        else:
            vmin = p5 * 2
            vmax = min(p95 * 2, nii_slice.max())
            circle_radius = 22

        if ser_modality in ('MRI T2', 'MRI T1post', 'CTA'):
            circle_radius = 12

        mask = np.ma.masked_where(seg_slice <= 0, seg_slice)
        fig, axes = plt.subplots(1, 2, figsize=(16, 8))

        axes[1].hist(nii_slice.flatten(), bins=50, color='skyblue', edgecolor='black')
        axes[1].axvline(x=vmin, color='red', linestyle='--')
        axes[1].axvline(x=vmax, color='red', linestyle='--')
        axes[1].set_title('Histogram of nii_slice Pixel Intensities')
        axes[1].set_xlabel('Pixel Intensity')
        axes[1].set_ylabel('Frequency')

        circle = plt.Circle((nii_k, nii_j), circle_radius, color='red', fill=False,)
        axes[0].add_patch(circle)
        axes[0].imshow(nii_slice, cmap='gray', vmin=vmin, vmax=vmax, interpolation='nearest')
        axes[0].imshow(mask, interpolation='nearest', alpha=None)
        axes[0].set_title(file_name)

        plt.tight_layout()
        #plt.show()
        plt.savefig(os.path.join(folder, f'{file_name}.png'))
        plt.close()


In [ ]:
'''import shutil

shutil.rmtree(folder)'''

In [ ]:
len(seg_with_loc)

In [ ]:
len(os.listdir(folder))

In [ ]:
folder_path = Path('/content/drive/Shareddrives/DATA/g143_review/D_Szyjka')
folder_path.mkdir(exist_ok=True)

hsv_save_folder = Path("/content/drive/Shareddrives/DATA/g143_review/D_Szyjka/dataset_zestackowane_slice'y_z_redukcja_nasycenia")
hsv_save_folder.mkdir(exist_ok=True)

img_save_folder = Path("/content/drive/Shareddrives/DATA/g143_review/D_Szyjka/dataset_zestackowane_slice'y")
img_save_folder.mkdir(exist_ok=True)

csv_path = folder_path / 'train.csv'

In [ ]:
def save_images_in_rgb_and_with_reduced_saturation(file_name, stacked3, rgb_path, hsv_path, vmin, vmax):
    normed = (stacked3 - vmin) / (vmax - vmin)
    normed = np.clip(normed, 0.0, 1.0)
    rgb_float = np.transpose(normed, (1, 2, 0))
    rgb = (rgb_float * 255).astype(np.uint8)

    Image.fromarray(rgb).save(os.path.join(rgb_path, f'{file_name}.png'))

    hsv = rgb_to_hsv(rgb_float)
    hsv[...,1] *= 0.1
    rgb_desat = hsv_to_rgb(hsv)
    rgb = (np.clip(rgb_desat,0,1) * 255).astype(np.uint8)

    Image.fromarray(rgb).save(os.path.join(hsv_path, f'{file_name}.png'))

In [ ]:
from PIL import Image
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb


jk_lst = []

for i, ser in enumerate(seg_with_loc):
    print(i)
    ser_modality = train_df.loc[train_df['SeriesInstanceUID'] == ser]['Modality'].iloc[0]

    sop_df = localizers.loc[localizers['SeriesInstanceUID'] == ser]
    nii_path = os.path.join(segmentations_path, f'{ser}.nii')
    nii_img = sitk.ReadImage(nii_path)
    nii_img = sitk.DICOMOrient(nii_img)
    nii_arr = sitk.GetArrayFromImage(nii_img)
    ser_i = []

    for idx, row in  sop_df.iterrows():
        slice_path = os.path.join(series_path, row.SeriesInstanceUID, f'{row.SOPInstanceUID}.dcm')
        img = sitk.ReadImage(slice_path)

        x, y = get_physical_point(img, row)
        x, y, z = np.array(img.TransformContinuousIndexToPhysicalPoint((x, y, 0)))

        nii_k, nii_j, nii_i = nii_img.TransformPhysicalPointToIndex((x, y, z))
        ser_i.append(nii_i)
        file_name = f'{ser}_{nii_i:04d}_{ser_modality}'
        jk_lst.append((file_name, nii_j, nii_k))

        stacked3 = nii_arr[nii_i-1 : nii_i+2, :, :]

        p5 = np.percentile(stacked3, 5)
        p95 = np.percentile(stacked3, 95)

        if ser_modality == 'CTA':
            vmin = -100
            vmax = 600
        else:
            vmin = p5 * 2
            vmax = min(p95 * 2, nii_slice.max())

        save_images_in_rgb_and_with_reduced_saturation(file_name, stacked3, img_save_folder, hsv_save_folder, vmin, vmax)

    min_i = min(ser_i)


    # brak tętniaka
    try:
        stacked3 = nii_arr[min_i-6 : min_i-3, :, :]

        if ser_modality != 'CTA':
            vmin = np.percentile(stacked3, 5)
            vmax = np.percentile(stacked3, 95)

        file_name = f'{ser}_{min_i-5:04d}_{ser_modality}'

        save_images_in_rgb_and_with_reduced_saturation(file_name, stacked3, img_save_folder, hsv_save_folder, vmin, vmax)

        jk_lst.append((file_name, 0, 0))

    except IndexError as e:
        print(f'\n{e} in file {file_name}\n')


df_jk = pd.DataFrame(jk_lst, columns=['file name', 'j', 'k'])
df_jk.to_csv(csv_path, index=False)

In [ ]:
'''import shutil

delete_path = Path("/content/drive/Shareddrives/DATA/g143_review/D_Szyjka/dataset_zestackowane_slice'y")
shutil.rmtree(delete_path)'''

In [ ]:
healthy_seg = 0

for idx, row in train_df.iterrows():
    if row.SeriesInstanceUID in localizers['SeriesInstanceUID']:
        continue

    if os.path.exists(os.path.join(segmentations_path, f'{row.SeriesInstanceUID}_cowseg.nii')):
        healthy_seg += 1

print(healthy_seg)


In [ ]:
len([f for f in os.listdir(segmentations_path)])

In [ ]:
localizers.shape

In [ ]:
train_df.shape